In [2]:
import re
import pandas as pd


In [3]:
# Load WhatsApp chat data from a text file
with open("whatsapp_chat.txt", "r", encoding="utf-8") as file:
    raw_data = file.readlines()

# Define the regex pattern to parse date, time, sender, and message
pattern = r"^(\d{1,2}/\d{1,2}/\d{2}),\s*(\d{1,2}:\d{2})\s*-\s*(.*?):\s*(.*)"

# Initialize a list to store extracted messages
data = []

# Temporary variables for multi-line message handling
current_message = ""
current_date = ""
current_time = ""
current_name = ""

# Process each line in the chat file
for line in raw_data:
    match = re.match(pattern, line)
    if match:
        # If line matches the pattern, it's a new message
        if current_message:
            # Save the previous message if exists
            data.append([current_date, current_time, current_name, current_message.strip()])
        
        # Extract components for the new message
        current_date = match.group(1)
        current_time = match.group(2)
        current_name = match.group(3)
        current_message = match.group(4)
    else:
        # If line does not match, assume it's a continuation of the previous message
        current_message += " " + line.strip()

# Save the last message if it exists
if current_message:
    data.append([current_date, current_time, current_name, current_message.strip()])

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data, columns=["Date", "Time", "Name", "Message"])

# Save to a CSV file or display the DataFrame
# df.to_csv("whatsapp_chat_parsed.csv", index=False)


In [4]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')

In [5]:
# Define the start date and end date (adjust as needed)
start_date = "04/12/24"
end_date = "09/12/24"

# Convert the start and end dates to datetime
start_date = pd.to_datetime(start_date, format='%d/%m/%y')
end_date = pd.to_datetime(end_date, format='%d/%m/%y')
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# Display the filtered DataFrame
df

,Date,Time,Name,Message
1758,2024-12-04,12:42,Sameeksha S DT,<Media omitted>
1759,2024-12-04,15:57,Paridhi Gupta DT,This message was deleted
1760,2024-12-04,18:14,Anay Pund,We are excited to announce last week's (25 Nov...
1761,2024-12-04,18:41,Sandeep Singh Kaintura DT,https://chat.whatsapp.com/IIuW2jty5BvDCLfbGgwl...
1762,2024-12-04,19:37,Paridhi Gupta DT,*Can gratitude change the way you lead your te...
...,...,...,...,...
1878,2024-12-09,21:30,Kamlesh Tak DT,07/12/2024 Reflection SD This weeks SD was st...
1879,2024-12-09,23:24,+91 77330 08964,"07/12/2024 Reflection SD Hello, I am Rakesh S..."
1880,2024-12-09,23:24,+91 77330 08964,07/12/2024 Self Gratitude I am grateful to my...
1881,2024-12-09,23:54,Shreya Sathyanarayanan DT,*07/12/2024 Reflection SD* In this week's Soc...


In [6]:
df.sample(5)

,Date,Time,Name,Message
1819,2024-12-07,13:30,Mohit Raj DT,"04/12/2024 Self Gratitude Thank you, Mohit, f..."
1882,2024-12-09,23:54,Shreya Sathyanarayanan DT,*07/12/2024 Self Gratitude* I am grateful for...
1808,2024-12-06,18:14,Janhvi Shukla DT,Hey all! We have a session planned for you t...
1771,2024-12-05,17:41,Paridhi Gupta DT,*What does it mean to think like a winner?* Gr...
1800,2024-12-06,17:34,Madhav Kumar DT,04/12/2024 Reflection LDI By understanding h...


In [7]:
contact_df = pd.read_csv("Contact pairs.csv")
contact_df.sample(3)

,Actual,Refined
57,+91 99379 58001,+91 99379 58001
70,Sayed Farisa DT,Sayed Farisa
62,+251 98 519 2183,+251 98 519 2183


In [8]:
contact_dict = {}
for i in range(len(contact_df)):
    contact_dict.update({contact_df.iloc[i, 0]: contact_df.iloc[i, 1]})

In [9]:
df['Name'] = df['Name'].apply(lambda x: x if x not in contact_dict else contact_dict[x])

In [10]:
df.sample(5)

,Date,Time,Name,Message
1841,2024-12-07,20:36,Paridhi Gupta,"hi everyone, please don't join the main lobby"
1843,2024-12-07,20:42,Artuha Paul,"The meeting has been locked, since the live SD..."
1774,2024-12-05,18:05,Sayed Farisa,"Hello everyone, We’re excited to invite you t..."
1838,2024-12-07,20:32,Artuha Paul,"Hi everyone, pls join townhall for Meta SD. Cc..."
1833,2024-12-07,19:31,Shreya Ghosh,<Media omitted>


## Classify data for LDI, SD and Gratitude or SD Pitch

In [11]:
# csv_path = input("Enter path to CSV file: ")
search = int(input("Scrapping\n1. LDI/SD/Gratitudes\ 2. SD Pitch:\n (1 or 2): "))
sundayDate = input("Enter the end date of week (DD/MM/YYYY): ")
if search == 1:
    search_words = ["SD", "LDI", "gratitude"]
elif search == 2:
    search_words = ["SD"]

for search_word in search_words:
    if search_word == "SD":
        df2 = df.copy()
        df2.drop(["Time", "Date"], axis="columns", inplace=True)

        df2["First_7_Words"] = df2["Message"].str.split().str[:7].str.join(" ")

        df2["flag"] = df2["First_7_Words"].str.contains(fr"\b{search_word}\b", case=False, regex=True)

        df3 = df2[df2["flag"] == True].drop(["Message", "First_7_Words"], axis="columns")

        # Transforming data for adaptability to system
        df3.drop(["flag"], axis="columns", inplace=True)
        df3['flag'] = "Yes"
        df3['Date'] = sundayDate

        print(f"Count: {df3.shape[0]}")
        
        if search == 1:
            sd_df = df3.copy()
            df3.to_csv(f'{search_word}.csv', header=False, index=False)
        else:
            sd_pitch_df = df3.copy()
            df3.to_csv(f'{search_word} Pitch.csv', header=False, index=False)
    elif search_word == "LDI":
        df2 = df.copy()
        df2.drop(["Time", "Date"], axis="columns", inplace=True)

        df2["First_7_Words"] = df2["Message"].str.split().str[:7].str.join(" ")

        df2["flag"] = df2["First_7_Words"].str.contains(fr"\b{search_word}\b", case=False, regex=True)

        df3 = df2[df2["flag"] == True].drop(["Message", "First_7_Words"], axis="columns")
        
        # Transforming data for adaptability to system
        df3.drop(["flag"], axis="columns", inplace=True)
        df3['flag'] = "Yes"
        df3['Date'] = sundayDate

        print(f"Count: {df3.shape[0]}")

        ldi_df = df3.copy()
        df3.to_csv(f'{search_word}.csv', header=False, index=False)
    elif search_word == "gratitude":
        df2 = df.copy()
        df2.drop(["Time", "Date"], axis="columns", inplace=True)

        df2["First_7_Words"] = df2["Message"].str.split().str[:7].str.join(" ")

        df2['Date'] = sundayDate
        df2["flag"] = df2["First_7_Words"].str.contains(fr"\b{search_word}\b", case=False, regex=True)

        df3 = df2[df2["flag"] == True].drop(["Message", "First_7_Words"], axis="columns")

        # Transforming data for adaptability to system
        df3.drop(["flag"], axis="columns", inplace=True)
        df3['flag'] = "TRUE"

        print(f"Count: {df3.shape[0]}")

        gratitude_df = df3.copy()
        df3.to_csv(f'{search_word}.csv', header=False, index=False)

Count: 18
Count: 30
Count: 45


In [13]:
import gspread

gc = gspread.service_account(filename='ubs-whatsapp-data-push-c191e1cd21aa.json')
spreadsheet = gc.open("UBS Prototype")


In [14]:
from gspread_dataframe import set_with_dataframe

In [15]:
def append_to_sheets(dataframes, subsheet_names):
    # Append each DataFrame to the corresponding subsheet
    for df, subsheet_name in zip(dataframes, subsheet_names):
        worksheet = spreadsheet.worksheet(subsheet_name)

        # Find the next available row in column B
        existing_data = worksheet.get_all_values()
        next_row = len(existing_data) + 1

        # Append DataFrame below existing data in column B
        set_with_dataframe(worksheet, df, row=next_row, col=2, include_index=False, include_column_header=False)

    print("Data appended successfully!")

Data appended successfully!


In [ ]:
if search_words == ["SD"]:
    append_to_sheets([sd_pitch_df], ["SD Pitch"])
else:
    append_to_sheets([gratitude_df, ldi_df, sd_df], ["Gratitudes", "LDI Reflections", "SD Reflections"])